In [1]:
import pandas as pd
import numpy as np

from data_processing import *
from utils import feature_desc

from sklearn.preprocessing import StandardScaler

#### Load data

In [2]:
df_og = load(csv_path='../data/merged.csv')

#### Aggregate time into clusters of size k

In [3]:
# k = 15
# df_og = aggregate(df_og, k)

### Proste featury wybieram albo tworze agregując


In [4]:
df = pd.DataFrame(index=df_og.index)

In [8]:
# srednia po 'WODY POWROTNE KOLEKTORÓW [°C]'
TIR = df_og.columns[df_og.columns.str.contains('tir')].map(lambda x: feature_desc(x)).values
df['TIR'] = df_og[TIR].mean(axis='columns')

KeyError: "None of [Index(['WODA POWROTNA KOLEKTORA KZ7 [°C]', 'WODA POWROTNA KOLEKTORA KZ8 [°C]',\n       'WODA POWROTNA KOLEKTORA KZ9 [°C]', 'WODA POWROTNA KOLEKTORA KZ10 [°C]',\n       'WODA POWROTNA KOLEKTORA KZ11 [°C]',\n       'WODA POWROTNA KOLEKTORA KZ12 [°C]',\n       'WODA POWROTNA KOLEKTORA KZ13 [°C]',\n       'WODA POWROTNA KOLEKTORA KZ15 [°C]'],\n      dtype='object')] are in the [columns]"

In [7]:
# srednia po 'TEMP POD 2 WARSTWĄ WYMURÓWKI [°C]'
TIX1 = df_og.columns[df_og.columns.str.contains('001tix')].values
df['TIX1'] = df_og[TIX1].mean(axis='columns')

KeyError: "None of [Index(['TEMP.1 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.2 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.3 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.4 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.5 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.6 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.7 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.8 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.9 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.10 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.11 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.12 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.13 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.14 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.15 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.16 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.17 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.18 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.19 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.20 POD 2 WARSTWĄ WYMURÓWKI [°C]',\n       'TEMP.21 POD 2 WARSTWĄ

In [ ]:
# prob_s i prob_corg
PR = ['prob_s', 'prob_corg']
df[PR] = df_og[PR]

In [ ]:
# reg nadawy koncentratu
FCX = df_og.columns[df_og.columns.str.contains('fcx')].map(lambda x: feature_desc(x)).values
df[FCX] = df_og[FCX]

In [ ]:
# sumaryczna moc cieplna
NIR = df_og.columns[df_og.columns.str.contains('nir')].map(lambda x: feature_desc(x)).values
df[NIR] = df_og[NIR]

In [ ]:
# WENT ODCZ ZAD OBROTÓW
UXM = df_og.columns[df_og.columns.str.contains('uxm')].values
df['UXM'] = df_og[UXM].mean(axis='columns')

In [ ]:
# dodanie temperatury żużla
df["TEMP_ZUZ"] = df_og["temp_zuz"]

In [ ]:
df

## Data is now ready to do somthing with it idk